In [12]:
import cv2 
import csv
import glob
import natsort
import numpy as np
import re
import pickle

In [13]:
def get_numbers_from_filename(filename):
    return re.search(r'\d+', filename).group(0)

In [14]:
def markSafeDurationForVideo(video_path):

    '''
    generates a list that contains safe duration(s) for video

    arguments:
    video_path(string) - video path for every video

    returns:
    safe_duration_list(list) - list containing safe duration start and safe duration end for the video     flag(boolean) - flag to indicate whether video is annotated successfully or not
    '''

    # fore resizing a window for video (optional depending on screen size)
    cv2.namedWindow(video_path, cv2.WINDOW_NORMAL)
    cv2.resizeWindow(video_path, height=760, width=1366)

    # setting font properties for displaying frame number with video
    font = cv2.FONT_HERSHEY_COMPLEX  # fontFace
    fontScale = 1  # fontScale
    fontColor = (255,255,255) # fontColor
    lineType = 2 # lineType
    
    # capture a video from given path
    scanned_video = cv2.VideoCapture(video_path)
    no_frames = int(scanned_video.get(cv2.CAP_PROP_FRAME_COUNT))
    
    frame_count = -1
    flag = 1

    ss = []  # safe duration start list
    se = [] # safe duration end list
    safe_duration_list = [] # final list containing ss and se

    if (scanned_video.isOpened()== False): # check if there is error in opening a video
        raise Exception("check the videopath or error opening video")

    while(scanned_video.isOpened()):

        ret, frame = scanned_video.read() 
        key = cv2.waitKey(33)
        
        if(ret == True):

            if(frame_count == -1):
                while True:

                    key_init = cv2.waitKey(33)
                    cv2.putText(frame, "Frame Number:" + str(frame_count), org=(20,50), fontFace=font, fontScale=fontScale, color=fontColor, lineType=lineType, thickness=3)
                    cv2.imshow(video_path, frame)

                    if key_init == ord('a'):  # to start a video when the video is loaded
                        break
            
            frame_count = frame_count + 1
            
            cv2.putText(frame, "Frame Number:" + str(frame_count), org=(20,50), fontFace=font, fontScale=fontScale, color=fontColor, lineType=lineType, thickness=3)
            cv2.imshow(video_path,frame)

            if key == ord('p'): # pause a video

                while True:
                    
                    key2 = cv2.waitKey(33)

                    cv2.putText(frame, "Frame Number:" + str(frame_count), org=(20,50), fontFace=font, fontScale=fontScale, color=fontColor, lineType=lineType, thickness=3)
                    cv2.imshow(video_path, frame)
                    
                    if key2 == ord('p'): # resume video after pausing
                        break
            
            if key == ord('r'): # key for the case when there is safe duration at the 0th frame(at the very start of the video)

                ss.append(0) # add 0th frame in list of safe duration start
                if (len(ss) != 1):
                    print("You have made some mistake")
                    flag = 0
                    break
                    
                safe_duration_list.append(0)
            
            if key == ord('s'): # save safe duration start

                ss.append(frame_count)
                if (len(ss) != len(se) + 1):
                    print("You have made some mistake")
                    flag = 0
                    break
                    
                safe_duration_list.append(frame_count)
                
            if key == ord('e'): # save safe duration end

                se.append(frame_count)
                if (len(se) != len(ss)):
                    print("You have made some mistake")
                    flag = 0
                    break
                safe_duration_list.append(frame_count)
            
            if key == ord('q'): # close video window
                break

        else:
            break


    scanned_video.release()
    cv2.destroyAllWindows()

    # if length of safe duration list is odd length, i.e. one safe duration end is left to be appended,      so add last frame of the video as safe duration end.
    if(len(safe_duration_list) % 2 !=0):
        safe_duration_list.append(no_frames - 1)
    
    return safe_duration_list, flag

In [15]:

def save_labels_csv(videos_list, csv_file, pickle_list_path):

    '''
    saves labels of safe durations from the videos into csv file

    arguments:
    videos_list(list) - list of all videos path stored as string
    csv_file(string) - path for csv file
    pickle_list_path - path for array pickle file

    '''
    
    csv_writer = csv.writer(open(csv_file,"w"))
    open_file = open(pickle_list_path, "rb")
    pickle_list = pickle.load(open_file)
    open_file.close()
    
    for video_path in videos_list:
        
        print('\nstarting for', video_path)
        video_no = int(get_numbers_from_filename(video_path)) # function for getting video number for current video
        ready_flag = str.lower(input('type "y" if you are ready to annotate the video and "n" to quit the program\n'))
        if(ready_flag == 'y'):
            correct_flag = 1
            surity_flag = 0
            while(surity_flag == 0 or correct_flag == 0): # loop through the video untill surity and correct flag is not valid
                safe_duration_list, correct_flag = markSafeDurationForVideo(video_path)
                if(correct_flag == 0):
                    print("press keys carefully this time!")
                    input("press any key when you are ready")
                else:
                    surity_flag = int(input('Enter 0 to repeat the annotations process and 1 to proceed to next video\n')) # for assuring that video is annotated as per needed by user
                if(surity_flag == 1 and correct_flag == 1):
                    break

            #csv_writer.writerow(safe_duration_list)
            pickle_list[video_no-1] = safe_duration_list # add list to pickle file with video number as index

            print("safe_duration_list", safe_duration_list)
            print('------------------------------------------')
            
        else:
            
            print('\nTake a break! You must be tired')
            break
            
    open_file = open(pickle_list_path, "wb")
    pickle.dump(pickle_list, open_file)
    open_file.close()
    csv_writer.writerows(pickle_list)
    print("list and csv file saved")

In [16]:
video_folder = '/home/yagnesh/Study/Machine Learning/ML projects/RoadCrossingAssistant_Arrays/videos'
video_list = glob.glob(video_folder + '/video*.MOV')
video_list = (natsort.natsorted(video_list))[80:]
pickle_list_path = '/home/yagnesh/Study/Machine Learning/ML projects/RoadCrossingAssistant_Arrays/labels_framewise.pkl'
# video_list = ['/home/yagnesh/Study/Machine Learning/ML projects/RoadCrossingAssistant_Arrays/videos_train/video1.MOV']

csv_file = '/home/yagnesh/Study/Machine Learning/ML projects/RoadCrossingAssistant_Arrays/labels-frame-wise.csv'

save_labels_csv(video_list, csv_file, pickle_list_path)


starting for /home/yagnesh/Study/Machine Learning/ML projects/RoadCrossingAssistant_Arrays/videos/video81.MOV
safe_duration_list [0, 94, 138, 257, 319, 394]
------------------------------------------

starting for /home/yagnesh/Study/Machine Learning/ML projects/RoadCrossingAssistant_Arrays/videos/video82.MOV
safe_duration_list [0, 87, 345, 448]
------------------------------------------

starting for /home/yagnesh/Study/Machine Learning/ML projects/RoadCrossingAssistant_Arrays/videos/video83.MOV
safe_duration_list [165, 375]
------------------------------------------

starting for /home/yagnesh/Study/Machine Learning/ML projects/RoadCrossingAssistant_Arrays/videos/video84.MOV
safe_duration_list [100, 151, 212, 331, 554, 611, 668, 749]
------------------------------------------

starting for /home/yagnesh/Study/Machine Learning/ML projects/RoadCrossingAssistant_Arrays/videos/video85.MOV
safe_duration_list [15, 104, 156, 226, 281, 490]
------------------------------------------

starti

In [17]:
#labels_framewise = [[] for i in range(104)]
file_name = "/home/yagnesh/Study/Machine Learning/ML projects/RoadCrossingAssistant_Arrays/labels_framewise.pkl"

#open_file = open(file_name, "wb")
#pickle.dump(labels_framewise, open_file)
#open_file.close()

open_file = open(file_name, "rb")
loaded_list = pickle.load(open_file)
open_file.close()

print(len(loaded_list),'\n', loaded_list)

104 
 [[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [70, 139], [0, 54], [16, 179], [0, 56, 103, 161], [], [24, 94, 140, 179], [21, 98], [134, 179], [69, 109], [66, 95, 117, 179], [], [129, 179], [34, 65], [], [], [0, 209], [208, 274], [0, 50, 170, 302, 366, 479], [0, 54, 108, 183, 234, 343], [32, 59, 273, 389], [0, 94, 138, 257, 319, 394], [0, 87, 345, 448], [165, 375], [100, 151, 212, 331, 554, 611, 668, 749], [15, 104, 156, 226, 281, 490], [12, 221], [117, 169, 228, 294, 394, 482], [0, 76, 157, 280, 430, 509], [168, 270], [97, 188, 284, 325, 362, 461, 496, 569], [0, 94, 362, 426], [276, 321, 357, 419], [0, 46, 67, 186], [16, 53, 211, 260, 284, 329], [10, 85, 144, 191, 271, 334], [15, 106, 279, 419], [0, 159, 243, 389], [], [37, 253, 300, 360], [0, 69, 132, 193, 268, 411], [246, 289, 324, 370]